In [1]:
import sys
#sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose')
#sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose/src')
#sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose/src/policies')
sys.path.append('/users/jclih/glucose')
sys.path.append('/users/jclih/glucose/src')
sys.path.append('/users/jclih/glucose/policies')

In [2]:
from policies.policies import model_smoothed_fitted_q
from Glucose import Glucose
from transition_estimation import MultivariateLinear
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [5]:
env = Glucose(horizon=100)
transition_model_fitter = MultivariateLinear
gamma = 0.9
number_of_value_iterations = 1

In [ ]:
done = False
env.reset()
while not done:
    action, pairwise_kernel_, kernel_sums = \
        model_smoothed_fitted_q(env, gamma, RandomForestRegressor, number_of_value_iterations, transition_model_fitter,
                                pairwise_kernels_=None, kernel_sums=None)
    _, r, done = env.step(action)

k 0 alpha_mb 1.0
k 0 alpha_mb 0.19780509935
k 0 alpha_mb 0.697334207704
k 0 alpha_mb 0.534223153088
k 0 alpha_mb 0.544308442866
k 0 alpha_mb 0.356007162818
k 0 alpha_mb 0.328668565836
k 0 alpha_mb 0.364727485761
k 0 alpha_mb 0.280233658185
k 0 alpha_mb 0.31461990545
k 0 alpha_mb 0.246090084497
k 0 alpha_mb 0.305849200952
k 0 alpha_mb 0.29316329096
k 0 alpha_mb 0.255319044423
k 0 alpha_mb 0.235959250663
k 0 alpha_mb 0.260415824411
k 0 alpha_mb 0.270444896696
k 0 alpha_mb 0.296222566372
k 0 alpha_mb 0.282434263905
k 0 alpha_mb 0.259925683881
k 0 alpha_mb 0.257117142834
k 0 alpha_mb 0.251238720529
